In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
events_data_train = pd.read_csv('https://stepik.org/media/attachments/course/4852/event_data_train.zip')
submissions_data_train = pd.read_csv('https://stepik.org/media/attachments/course/4852/submissions_data_train.zip')

In [3]:
events_data_train.head()

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632


In [5]:
submissions_data_train.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [8]:
events_data_train['date'] = pd.to_datetime(events_data_train['timestamp'], unit='s')
events_data_train.head()

,step_id,timestamp,action,user_id,date
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48
1,32815,1434340848,passed,17632,2015-06-15 04:00:48
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35


In [10]:
events_data_train['day'] = events_data_train.date.dt.date
events_data_train.head()

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15


In [14]:
events_data_train.pivot_table(index='user_id',
                              columns='action',
                              values='step_id',
                              aggfunc='count'
                             )\
    .reset_index()\
    .sort_values(by='started_attempt', ascending=False)\
    .head(10)

action,user_id,discovered,passed,started_attempt,viewed
781,1046,128.0,124.0,721.0,8122.0
2611,3572,194.0,193.0,550.0,6968.0
6062,8394,131.0,94.0,337.0,1070.0
4825,6662,198.0,197.0,285.0,1054.0
1204,1649,194.0,194.0,281.0,807.0
5565,7702,187.0,115.0,276.0,1328.0
3938,5454,195.0,195.0,265.0,1167.0
3700,5096,184.0,178.0,232.0,1608.0
15512,21602,191.0,191.0,227.0,408.0
4007,5552,198.0,198.0,224.0,1141.0
